# 解析方針

- 局所冷却のみでなく，Nifekalantのデータや，controlのデータに対しても累積位相分散解析をしてみる
- まずデータに関して下調べする必要あり

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import cv2
import copy
from scipy import ndimage
import glob
%matplotlib inline

In [2]:
from opmap.RawCam import RawCam
from opmap.VmemMap import VmemMap
from opmap.PhaseMap import PhaseMap
from opmap.PhaseVarianceMap import PhaseVarianceMap
from opmap.APDMap import APDMap

# 局所冷却データ(2006-02-17)

In [3]:
paths = glob.glob('/mnt/recordings/ExperimentData/2006-02-17/*')
paths.pop(1)

'/mnt/recordings/ExperimentData/2006-02-17/C001S0027'

In [ ]:
threshold = 0.4

for path in paths:
    print(path)
    
    output_path = path.replace('ExperimentData', 'AnalysisResult/Seno/opmap/HRS2018/20171014')
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    cam = RawCam(path=path,
                 cam_type='sa4',
                 image_width=256, image_height=256,
                 frame_start=0, frame_end=-1)
    cam.setIntROI(val_min=100)
    cam.saveImage(os.path.join(output_path, 'cam'), skip=10000)
    
    vmem = VmemMap(cam)
    vmem.smooth(size=20)
    vmem.saveImage(os.path.join(output_path, 'vmem'), skip=10)
    
    pmap = PhaseMap(vmem, shrink=2)
    pmap.saveImage(os.path.join(output_path, 'pmap'), skip=10)

    pvmap = PhaseVarianceMap(pmap, size=3)
    pvmap.saveImage(os.path.join(output_path, 'pvmap'), skip=10)

    """
    APD = APDMap(vmem)
    APD.saveImage(os.path.join(output_path, 'APD'), skip=10)
    """
    
    APV_array = np.zeros((128, 128))
    phase_variance_array = copy.deepcopy(pvmap.data)
    ROI = copy.deepcopy(pvmap.roi)
    core = (phase_variance_array > threshold) * 1
    core_count = np.sum(core, axis=0)
    
    for i in range(128):
        for j in range(128):
            if ROI[i,j] != 0:
                data = copy.deepcopy(phase_variance_array[:, i, j])
                time = data.shape[0]
                while np.max(data) > threshold:
                    x_pos = np.argmax(data)
                    x_max = x_pos + 1
                    x_min = x_pos - 1
                    while x_max < time and data[x_max - 1] > data[x_max]:
                        x_max += 1
                    while x_min > -1 and data[x_min] < data[x_min + 1]:
                        x_min -= 1
                    data[x_min+2:x_max] = 0
                APV_array[i,j] = np.sum(data)/len(np.nonzero(data)[0])

    plt.imshow(core_count, cmap='jet')
    plt.colorbar()
    plt.savefig(os.path.join(output_path, 'core.png'), dpi=300, transparent=True)
    plt.close()
    
    plt.imshow(APV_array, cmap='jet')
    plt.colorbar()
    plt.savefig(os.path.join(output_path, 'APV.png'), dpi=300, transparent=True)
    plt.close()

/mnt/recordings/ExperimentData/2006-02-17/C001S0024
/mnt/recordings/ExperimentData/2006-02-17/C001S0007
/mnt/recordings/ExperimentData/2006-02-17/C001S0015
